In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"GPU available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU name: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.5.1
GPU available: True
Number of GPUs: 1
Current GPU name: NVIDIA RTX A6000


In [4]:
import pandas as pd

# 데이터 파일 경로 설정
data_path = 'Esperanto-Dataset/Dataset/News_AI.csv'
df = pd.read_csv(data_path)

# DataFrame에 어떤 열(column)들이 있는지 출력해봅니다.
print("Actual column names in the file:")
print(df.columns)

Actual column names in the file:
Index(['LLM', 'AI', 'PT', 'ES', 'FR', 'IT', 'ZH', 'NL', 'DA', 'JA', 'DE',
       'KO'],
      dtype='object')


In [5]:
# 위에서 확인한 올바른 열 이름을 여기에 넣어주세요.
# 예: text_column_name = 'article'
text_column_name = 'LLM'

# 상위 5개 텍스트 샘플 출력
print("\nAI-generated text samples from the dataset:")
for text in df[text_column_name].head():
    print("-" * 50)
    print(text)


AI-generated text samples from the dataset:
--------------------------------------------------
Mistral 7b
--------------------------------------------------
Mistral 7b
--------------------------------------------------
Mistral 7b
--------------------------------------------------
Mistral 7b
--------------------------------------------------
Mistral 7b


In [6]:
import pandas as pd

# 데이터 파일 경로 설정
data_path = 'Esperanto-Dataset/Dataset/News_AI.csv'
df = pd.read_csv(data_path)

# 텍스트가 들어있는 실제 열 이름은 'AI' 입니다.
text_column_name = 'AI'

# 상위 5개 텍스트 샘플 출력
print("Original AI-generated text samples from the dataset:")
for text in df[text_column_name].head():
    print("-" * 50)
    print(text)

Original AI-generated text samples from the dataset:
--------------------------------------------------
Washington D.C. - In a surprising turn of events, President Donald Trump disinvited the Philadelphia Eagles from the Super Bowl champions ceremony at the White House on Tuesday, citing a disagreement over standing for the national anthem. However, none of the Eagles' players took a knee during the 2017 season.  In response to the disinvitation, Philadelphia Mayor Jim Kenney released a statement criticizing Trump's decision, labeling him a "fragile egomaniac." Fewer than 10 Eagles players were reportedly committed to attending the White House event, making the decision to disinvite the team a largely symbolic one.  Instead of welcoming the Super Bowl champions to the White House, the President held a "Celebration of America" event. The event featured military bands and a group singing of "God Bless America." Trump reiterated his strong support for the country, the flag, and the nation

In [ ]:
# ==============================================================================
# Step 1: 필요한 도구들 불러오기 (Setup)
# ==============================================================================
import os
import deepl
import pandas as pd
from dotenv import load_dotenv
from tqdm.notebook import tqdm
from google.cloud import translate_v2 as translate # Google Translate 라이브러리
import requests # LibreTranslate API 호출을 위한 라이브러리
import time # API 호출 사이에 약간의 지연을 주기 위한 도구

# ==============================================================================
# Step 2: 실험 조건 설정하기 (Configuration)
# ==============================================================================
# --- API 키 설정 ---
load_dotenv()
DEEPL_API_KEY = os.getenv('BTcJ3KUxsMjgF5mSv')

# --- 실험 변수 설정 ---
# 여기서 우리 실험의 모든 조건을 제어합니다.
# ★★★★★ 테스트하고 싶은 번역기 이름을 이 리스트에 추가하세요 ★★★★★
engines_to_test =

# 우선은 '계통적으로 가까운 언어'인 독일어(DE) 하나만 테스트
pivot_language = 'DE'

# 우선은 1회 반복만 테스트
iterations = 1

# 실험할 텍스트 개수 (10개로 빠르게 테스트)
num_samples_to_test = 10

# 원본 데이터 파일 경로
source_data_path = 'Esperanto-Dataset/Dataset/News_AI.csv'

# ==============================================================================
# Step 3: 각 번역기별 역번역 함수 만들기 (Helper Functions)
# ==============================================================================
# 각 번역기마다 API 사용법이 다르므로, 각각에 맞는 함수를 만들어 줍니다.

# --- DeepL 번역 함수 ---
def back_translate_deepl(text, pivot_lang, num_iterations):
    translator = deepl.Translator(DEEPL_API_KEY)
    current_text = text
    try:
        for _ in range(num_iterations):
            to_pivot = translator.translate_text(current_text, target_lang=pivot_lang).text
            time.sleep(1) # API 과부하 방지를 위해 1초 대기
            to_english = translator.translate_text(to_pivot, target_lang="EN-US").text
            current_text = to_english
        return current_text
    except Exception as e:
        return f"Error: {e}"

# --- Google Translate 번역 함수 ---
def back_translate_google(text, pivot_lang, num_iterations):
    client = translate.Client()
    current_text = text
    try:
        for _ in range(num_iterations):
            to_pivot = client.translate(current_text, target_language=pivot_lang)
            time.sleep(1) # API 과부하 방지를 위해 1초 대기
            to_english = client.translate(to_pivot, target_language='en')
            current_text = to_english
        return current_text
    except Exception as e:
        return f"Error: {e}"

# --- LibreTranslate 번역 함수 ---
def back_translate_libre(text, pivot_lang, num_iterations):
    url = "https://libretranslate.de/translate"
    current_text = text
    try:
        for _ in range(num_iterations):
            # 영어 -> 피봇 언어
            params_to_pivot = {"q": current_text, "source": "en", "target": pivot_lang.lower()}
            response_to_pivot = requests.post(url, json=params_to_pivot)
            to_pivot = response_to_pivot.json()
            
            time.sleep(1) # API 과부하 방지를 위해 1초 대기

            # 피봇 언어 -> 영어
            params_to_english = {"q": to_pivot, "source": pivot_lang.lower(), "target": "en"}
            response_to_english = requests.post(url, json=params_to_english)
            to_english = response_to_english.json()
            current_text = to_english
        return current_text
    except Exception as e:
        return f"Error: {e}"

# ==============================================================================
# Step 4: 실제 실험 실행하기 (Main Experiment Loop)
# ==============================================================================
# 이제 위에서 설정한 모든 번역기에 대해 실험을 자동으로 실행합니다.

# 1. 원본 데이터 불러오기
print(f"Loading original data from {source_data_path}...")
original_df = pd.read_csv(source_data_path)
original_texts = original_df['AI'].head(num_samples_to_test).tolist()
print(f"Loaded {len(original_texts)} text samples for the test run.")

# 2. 전체 결과를 저장할 리스트 만들기
all_results =

# 3. 설정된 모든 번역기에 대해 루프 실행
for engine_name in engines_to_test:
    print(f"\n===== Starting experiment for engine: {engine_name} =====")
    
    # tqdm으로 진행 상황 표시
    for original_text in tqdm(original_texts, desc=f"Translating with {engine_name}"):
        translated_text = None
        # 엔진 이름에 따라 올바른 함수를 호출합니다.
        if engine_name == 'DeepL':
            translated_text = back_translate_deepl(original_text, pivot_lang=pivot_language, num_iterations=iterations)
        elif engine_name == 'Google':
            translated_text = back_translate_google(original_text, pivot_lang=pivot_language, num_iterations=iterations)
        elif engine_name == 'LibreTranslate':
            translated_text = back_translate_libre(original_text, pivot_lang=pivot_language, num_iterations=iterations)
        
        # 결과를 딕셔너리 형태로 저장
        all_results.append({
            'original_text': original_text,
            'translated_text': translated_text,
            'pivot_language': pivot_language,
            'iterations': iterations,
            'engine': engine_name
        })

print("\n===== All translation processes completed. =====")

# 4. 최종 결과를 하나의 CSV 파일로 저장하기
results_df = pd.DataFrame(all_results)
output_filename = f"all_engines_results_{pivot_language}_{iterations}_iter.csv"
results_df.to_csv(output_filename, index=False)

print(f"\nSuccessfully saved all results to '{output_filename}'")
print("\n--- First 10 rows of the combined result ---")
print(results_df.head(10))

SyntaxError: invalid syntax (72840620.py, line 79)